In [197]:
pip install --upgrade joblib


Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import joblib

# Load the saved models
clf_pipeline = joblib.load('classification_model.pkl')
reg_model = joblib.load('regression_model.pkl')
scaler = joblib.load('scaler.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')  # Load the saved TF-IDF vectorizer

# Load the Word2Vec model
word2vec_model = Word2Vec.load('word2vec_model.pkl')

# Load the test data
test_df = pd.read_csv('cleanedData.csv')
print(test_df)
# Handle missing values in text columns
text_columns = ['girisimcilikle_ilgili_deneyiminiz_var_mi?', 'girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?']
test_df[text_columns] = test_df[text_columns].fillna('')

# Combine text columns into one for processing
test_df['combined_text'] = test_df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize and preprocess text data for Word2Vec
def preprocess_text(text):
    return simple_preprocess(text, deacc=True)

# Create document vectors by averaging word vectors
def vectorize_text(text):
    tokens = preprocess_text(text)
    vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
    if len(vectors) == 0:
        return np.zeros(word2vec_model.vector_size)
    return np.mean(vectors, axis=0)

# Vectorize text data
test_df['text_vector'] = test_df['combined_text'].apply(vectorize_text)
word2vec_features_test = np.array(test_df['text_vector'].tolist())

# Vectorize test data with the fitted TF-IDF vectorizer
tfidf_vectors_test = tfidf_vectorizer.transform(test_df['combined_text'])

# Combine Word2Vec and TF-IDF features
combined_features_test = hstack([tfidf_vectors_test, word2vec_features_test])

# Prepare data for classification model
X_cat_test = test_df[cat_features]  # Ensure cat_features is defined and matches your training data

# Predict using classification model
y_pred_cat = clf_pipeline.predict(X_cat_test)
print("Classification Predictions:\n", y_pred_cat)

# Prepare data for regression model
X_combined_test = combined_features_test
X_combined_test = scaler.transform(X_combined_test)

# Predict using regression model
y_pred_reg = reg_model.predict(X_combined_test)
print("Regression Predictions:\n", y_pred_reg)


      Unnamed: 0  basvuru_yili  cinsiyet dogum_yeri ikametgah_sehri  \
0              0          2023         1     Amasya          Amasya   
1              1          2023         0     Samsun        İstanbul   
2              2          2023         0    isparta           İzmir   
3              3          2023         0      Hatay        Kirsehir   
4              4          2023         1   Adiyaman        Adiyaman   
...          ...           ...       ...        ...             ...   
9936        9936          2023         1     Ankara          Ankara   
9937        9937          2023         1     Yozgat          Ankara   
9938        9938          2023         1      Konya           Konya   
9939        9939          2023         1   Adiyaman        İstanbul   
9940        9940          2023         1      Nigde          Ankara   

                          universite_adi universite_turu  \
0                    Munzur universitesi          Devlet   
1                  Bogazici